In [1]:

# %%

import imghdr
from IPython.display import display_html
import os
import plotly.graph_objects as go
import dataframe_image as dfi

from urllib.request import urlopen
import plotly.express as px
import entsoe
from entsoe import EntsoePandasClient
import pandas as pd
from pandas import json_normalize
import pandas as pd
from datetime import timedelta, date
import datetime as dt
import time
from plotly.graph_objs import layout
import pytz
from folium import plugins
import smtplib
import plotly
import html
import mimetypes

from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display, HTML
import smtplib

import plotly.graph_objs as gobj
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

pd.options.plotting.backend = "plotly"
pd.set_option('precision', 1)

# %%
client = EntsoePandasClient(api_key='31a5379a-3e8c-4309-81ba-00e14636b8fb')

country_codes_list = ['ES', 'FR', 'DE_LU', 'IT_CNOR', 'NL', 'PL', 'BE', 'HU', 'CZ', 'HR', 'FI',
                      'GR', 'NO_1', 'PT', 'SE_4', 'AT', 'DK_1', 'EE', 'LT', 'LV', 'BG', 'RO', 'RS', 'SK', 'SI']

type_marketagreement_type = 'A01'
contract_marketagreement_type = 'A01'

start = pd.Timestamp(dt.datetime.now()+timedelta(days=-7),
                     tz='Europe/Brussels')
startdate = start.replace(hour=00, minute=00, second=0, microsecond=0)

end = pd.Timestamp(dt.datetime.now()+timedelta(days=1), tz='Europe/Brussels')
enddate = end.replace(hour=23, minute=00, second=0, microsecond=0)

df_countries = []
for x in country_codes_list:
    df = pd.DataFrame(client.query_day_ahead_prices(
        country_code=x, start=startdate, end=enddate), columns=[x])
    df_countries.append(df)

df_countries = pd.concat(df_countries, axis=1)
df_countries.index = df_countries.index.tz_convert('Europe/Brussels')
df_countries_table = df_countries
df_countries = df_countries.rename(columns={'ES': 'Spain', 'FR': 'France', 'DE_LU': 'Germany', 'NL': 'Netherlands', 'PL': 'Poland', 'BE': 'Belgium', 'HU': 'Hungary', 'CZ': 'Czech Republic', 'HR': 'Croatia', 'FI': 'Finland', 'GR': 'Greece', 'NO_1': 'Norway',
                                   'PT': 'Portugal', 'SE_4': 'Sweden', 'AT': 'Austria', 'DK_1': 'Denmark', 'IT_CNOR': 'Italy', 'EE': 'Estonia', 'LT': 'Lithuania', 'LV': 'Latvia', 'BG': 'Bulgaria', 'RO': 'Romania', 'RS': 'Serbia', 'SK': 'Slovakia', 'SI': 'Slovenia'})
df_daily = df_countries.groupby(df_countries.index.date).mean()
df_da = df_daily.tail(1)
df_da_map = df_da.T
df_da_map = df_da_map.reset_index()
df_da_map.columns = ['CountryName', 'da_price']

# %%
df_countries_table = df_countries_table.groupby(
    df_countries_table.index.date).mean()
#df_countries_table = df_countries_table.tail(1)
#df_countries_table = df_countries_table.iloc[:,:15]

# %%
enddate = pd.to_datetime(enddate, format='%Y-%m-%d %H:%M', utc=True)
enddate = enddate.strftime('%d/%m/%Y')

data = dict(type='choropleth',
            locations=df_countries.columns,
            locationmode='country names',
            colorscale='magma',
            text=df_da_map['CountryName'],
            z=df_da_map['da_price'],
            colorbar={'title': '€/MWh', 'len': 250, 'lenmode': 'pixels'},
            )

layout = dict(geo={'scope': 'europe'})

col_map = gobj.Figure(data=[data], layout=layout)

col_map.update_layout(height=750, width=750, margin=dict(
    l=1, r=1, t=50, b=1), title='Daily Power Prices as of '+str(enddate))


cm = sns.color_palette('magma', as_cmap=True)
df_countries_table = df_countries_table.style.background_gradient(
    cmap=cm, axis=1)
#display(iplot(col_map)
#col_map.write_image("yourfile.png")
col_map.write_image("eu_map.png")
dfi.export(df_countries_table, 'eu_table.png')
print('Attachments are done')


display(df_countries_table)
display(iplot(col_map))


Attachments are done


,ES,FR,DE_LU,IT_CNOR,NL,PL,BE,HU,CZ,HR,FI,GR,NO_1,PT,SE_4,AT,DK_1,EE,LT,LV,BG,RO,RS,SK,SI
2022-04-04,251.9,551.4,75.3,348.9,212.3,91.2,225.8,230.4,176.9,217.9,69.7,334.4,172.8,251.4,66.1,209.6,75.3,72.4,72.4,72.4,228.2,232.7,223.3,220.4,217.9
2022-04-05,265.0,308.3,183.0,305.4,239.2,109.0,184.6,216.8,194.0,220.2,52.8,278.1,181.7,265.0,74.7,217.3,153.6,83.4,83.4,83.4,209.9,208.5,222.9,202.5,220.2
2022-04-06,255.0,274.4,121.3,284.7,185.7,117.4,127.7,174.6,145.5,180.1,81.8,282.9,173.9,255.0,113.4,168.4,121.3,119.1,119.1,119.1,172.1,174.6,193.3,167.3,180.1
2022-04-07,214.8,219.5,74.2,244.6,200.6,87.6,82.0,168.0,98.1,159.9,78.0,230.3,170.1,214.8,78.6,140.2,76.6,85.1,85.1,85.1,169.6,168.0,160.4,142.7,159.9
2022-04-08,154.7,248.1,120.9,281.6,228.7,97.0,212.9,162.1,125.7,211.9,28.6,247.0,170.6,154.7,41.6,152.5,107.2,47.0,47.0,47.0,158.5,159.2,181.1,140.0,211.9
2022-04-09,241.6,241.6,59.8,232.7,180.9,85.4,133.5,134.7,78.9,118.1,28.1,228.9,166.0,241.6,28.1,116.5,59.8,29.8,29.8,29.8,133.4,134.7,131.3,117.1,118.1
2022-04-10,159.3,217.9,74.7,196.2,189.2,59.3,182.7,90.2,79.4,94.5,18.9,168.2,167.8,159.3,18.9,92.0,62.8,20.6,20.6,20.6,83.0,83.6,126.0,81.7,94.5
2022-04-11,163.1,229.0,223.6,238.2,233.1,133.4,233.2,227.6,223.6,225.3,76.7,243.1,190.2,163.1,143.5,225.2,218.9,138.3,144.7,138.3,179.5,172.0,238.5,227.6,225.3
2022-04-12,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,222.0,nan,nan


None

Email sent
